In [62]:
import numpy as np 
import matplotlib.pyplot as plt 
import sklearn.neighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import pytesseract
from PIL import Image
import cv2
import pandas as pd 
import skimage
from pathlib import Path

## Dataset paths and loading

In [63]:
from common import *

## Load sample image

In [64]:
image_id = 1
im, data = load_sample(image_id)

## Drawing funcs

In [65]:
# Linetypes
def draw_pipelines(image, data=data):
    draw = image.copy()
    solid_lines = np.stack(data["lines"].query("type=='solid'")["box"])
    dashed_lines = np.stack(data["lines"].query("type=='dashed'")["box"])

    draw = cv2.drawContours(draw, solid_lines.reshape(-1,2,2), -1, (255, 255, 0), thickness=2)
    draw = cv2.drawContours(draw, dashed_lines.reshape(-1,2,2), -1, (0, 255, 255), thickness=2)
    return draw

def draw_detections(image, rects, classes,color=(255,0,0)):
    draw_rects(image, rects, thickness=8, color=color)
    for r,c in zip(rects,classes):
        cv2.putText(image, str(c), r.flatten()[:2], cv2.FONT_HERSHEY_PLAIN, 6, (0,0,255))

def draw_symbols(image, data=data, color=None, thickness=2):
    draw = image.copy()
    for i, group in data["symbols"].groupby("class"):
        color_ = color or (np.random.rand(3)*255).astype(np.uint8)
        symbols = np.stack(group["box"])
        draw_rects(draw, symbols, color=[int(c) for c in color_], thickness=thickness)
    return draw

def draw_gt_symbols(image, data=data,color=None, thickness=2):
    symbol_boxes = np.stack(data["symbols"]["box"])
    symbol_classes = np.stack(data["symbols"]["class"]).astype(int)
    draw_detections(image, symbol_boxes, symbol_classes, color=color)

def draw_text_boxes(image, data=data, color=(255,0,255), thickness=1):
    draw = image.copy()
    text_boxes = np.stack(data["words"]["box"])
    draw_rects(draw, text_boxes, color=color, thickness=thickness)
    return draw


In [66]:
%matplotlib qt


# im = cv2.imread("test.jpg")
draw = im.copy()
draw = draw_pipelines(draw)
draw = draw_symbols(draw)
draw = draw_text_boxes(draw)
plt.imshow(draw)

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\admpdi\Code\su-visao\WPy64-3880\python-3.8.8.amd64\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\admpdi\Code\su-visao\WPy64-3880\python-3.8.8.amd64\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "c:\Users\admpdi\Code\personal\CMP197\pid_detect\.venv\lib\site-packages\matplotlib\backends\_backend_tk.py", line 489, in delayed_destroy
    self.window.destroy()
  File "C:\Users\admpdi\Code\su-visao\WPy64-3880\python-3.8.8.amd64\lib\tkinter\__init__.py", line 2312, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed


In [67]:
gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

plt.imshow(thresh)
plt.tight_layout()

In [68]:
X = np.stack(data['words']["box"])

whs = np.abs(np.stack((X[:,0] - X[:,2], X[:,1]-X[:,3]))).T

text_heights = np.min(whs,axis=1)
print("Average text height:")
np.mean(text_heights[text_heights>0])

Average text height:


35.60747663551402

## Detection with blackhat

In [69]:
def detect_symbols(image):

    if image.ndim == 3:
        gray = np.mean(image,axis=-1).astype(np.uint8)
    else:
        gray=image

    t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Foreground is smaller than 50% of image
    if np.count_nonzero(thresh) > thresh.size/2:
        thresh = 255-thresh

    skel = skimage.morphology.skeletonize(thresh//255, method="lee")

    kern = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(41,41))
    closing_kern = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(5,5))

    blackhat = cv2.morphologyEx(skel, cv2.MORPH_BLACKHAT, kern)

    blackhat = cv2.morphologyEx(blackhat, cv2.MORPH_OPEN, closing_kern)
    blackhat = cv2.morphologyEx(blackhat, cv2.MORPH_CLOSE, closing_kern)

    # blackhat = cv2.morphologyEx(blackhat, cv2.MORPH_DILATE, closing_kern, iterations=2)
    # blackhat = cv2.morphologyEx(blackhat, cv2.MORPH_ERODE, closing_kern, iterations=1)

    contours, hierarchy = cv2.findContours(blackhat*255, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    symbol_boxes = []
    for c in contours:
        if cv2.contourArea(c) > 10:
            x,y,w,h =cv2.boundingRect(c)
            symbol_boxes.append([x,y,x+w,y+h])

    return np.stack(symbol_boxes)


## Detect without text removal

In [70]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

draw = im.copy()

symbol_boxes = detect_symbols(im)

draw_rects(draw, np.stack(symbol_boxes).reshape(-1,2,2), thickness=8)
axs[0].imshow(draw)

draw2 = draw_symbols(draw, thickness=8, color=(0,255,0))
axs[1].imshow(draw2)

## Detect with text removal

In [71]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

# Pre-process
im_cleanup = cleanup_text(im, img_id=image_id)
draw = im_cleanup.copy()

# Detect
symbol_boxes = detect_symbols(im_cleanup)

# Draw results
draw_rects(draw, np.stack(symbol_boxes).reshape(-1,2,2), thickness=8)
axs[0].imshow(draw)
draw2 = draw_symbols(draw, thickness=8, color=(0,255,0))
axs[1].imshow(draw2)

## Classsification

### Features for each object

In [72]:
import mahotas 

def get_largest_contour(im):
    contours, hierarchy = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cmax = sorted(contours, key=cv2.contourArea)[-1]
    return cmax

def zernike_adaptive_centroid(image, degree=8):
    c = get_largest_contour(image)
    (x,y),r = cv2.minEnclosingCircle(c)
    return  mahotas.features.zernike_moments(image, r, degree=degree)

def rect_to_slice(rect_pts, margin=0):
    """
    Convert cv-style rect to numpy-style slice
    """
    (x0, y0), (x1, y1) = rect_pts

    return (slice(y0-margin, y1+margin), slice(x0-margin, x1+margin))

In [73]:
def get_square_thresh(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    return resizeAndPad(thresh, (64,64), padColor=0)

def get_zernike_features(image):
    thresh = get_square_thresh(image)
    return zernike_adaptive_centroid(thresh)

crops = [ im_cleanup[rect_to_slice(s.reshape(2,2), margin=15)] for s in symbol_boxes] 
features = [ get_zernike_features(crop) for crop in crops]

print(np.stack(features))

[[0.31830989 0.01106598 0.38703246 ... 0.05597141 0.03592777 0.02536582]
 [0.31830989 0.0021187  0.37050018 ... 0.08079489 0.13086272 0.03363618]
 [0.31830989 0.00048755 0.35029793 ... 0.04180301 0.09806796 0.01287733]
 ...
 [0.31830989 0.00225107 0.41444083 ... 0.03071122 0.0328579  0.08483224]
 [0.31830989 0.0056194  0.26216208 ... 0.06943625 0.19528019 0.16530438]
 [0.31830989 0.02421316 0.1962177  ... 0.13619354 0.02213319 0.04046806]]


#### Clustering

In [26]:
import sklearn.cluster

centroid, labels, _ = sklearn.cluster.k_means(np.stack(features), n_clusters=5)
# b = sklearn.cluster.estimate_bandwidth(np.stack(features))
# centroid, labels = sklearn.cluster.mean_shift(np.stack(features), bandwidth=b)

NameError: name 'features' is not defined

In [ ]:
for l in np.unique(labels):
    fig,ax = plt.subplots(1, 1+np.count_nonzero(labels==l))
    i=0
    for label, crop in zip(labels,crops):
        if label==l:
            ax[i].imshow(crop)
            i+=1

### Training split

In [29]:
import re

def load_training_set(feature_func):
    training_img_ids = range(400,500) 
    templates_path = Path("../templates")

    file_pattern = re.compile(r"im(\d+)_sym(\d+)")

    train_x = []
    train_y = []
    for template in templates_path.glob("**/*.png"):
        file = template.stem
        id, sym = file_pattern.match(file).groups() 

        if int(id) in training_img_ids:
            template_im = cv2.imread(str(template))

            # Features
            train_x.append(feature_func(template_im))
            # Folder is the class
            train_y.append(int(template.parent.stem))

    return np.stack(train_x), np.stack(train_y)


In [74]:
def eval_results(image, boxes, feature_func, class_pipeline):
    fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

    draw = image.copy()
    draw2 = image.copy()

    # Detect
    # symbol_boxes = detect_symbols(im_cleanup)
    # symbol_boxes = np.stack(data["symbols"]["box"])
    crops = [ image[rect_to_slice(s.reshape(2,2), margin=15)] for s in boxes] 
    features = [ feature_func(crop) for crop in crops]

    probas = class_pipeline.predict_proba(features)
    predictions = 1+np.argmax(probas,axis=1)
    confidences = np.max(probas,axis=1)

    draw_gt_symbols(draw2, thickness=8, color=(0,255,0))
    axs[0].imshow(draw2)
    axs[0].set_title("$\it{Ground Truth}$")

    # Draw results
    draw_detections(draw, np.stack(boxes).reshape(-1,2,2), predictions)
    axs[1].imshow(draw)
    axs[1].set_title("Detectado")

    gt_boxes = np.stack(data["symbols"]["box"])
    gt_classes = np.stack(data["symbols"]["class"]).astype(int)

    metric_data = []
    for clas in np.unique(gt_classes):
        gt = gt_boxes[gt_classes==clas]
        pred = boxes[predictions==clas]
        metrics = detection_metrics(gt,pred)
        metric_data.append({"clas":clas, "precision":metrics[0], "recall":metrics[1]})
    df = pd.DataFrame(metric_data)
    return df


In [31]:
pipe = Pipeline([("scaler", StandardScaler()), ("classifier", sklearn.neighbors.KNeighborsClassifier(n_neighbors=5))])

train_x, train_y = load_training_set(feature_func=get_zernike_features)
pipe.fit(train_x, train_y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', KNeighborsClassifier())])

In [57]:
df = eval_results(image=im_cleanup, 
             boxes=detect_symbols(im_cleanup),
             feature_func=get_zernike_features,
             class_pipeline=pipe)

df

c:\Users\admpdi\Code\personal\CMP197\pid_detect\notebook\common.py:330: RuntimeWarning: invalid value encountered in long_scalars
  recall = np.sum(TP) / (np.sum(FN) + np.sum(TP))


,clas,precision,recall
0,1,0.000000,0.000000
1,2,1.000000,0.500000
2,3,1.000000,0.500000
3,4,1.000000,0.333333
4,5,0.666667,0.666667
5,6,0.250000,1.000000
6,7,1.000000,0.307692
7,9,0.500000,0.666667
8,10,0.750000,0.857143
9,11,1.000000,0.800000


In [75]:
import sklearn.svm

pipe = Pipeline([("scaler", StandardScaler()), ("classifier", sklearn.svm.SVC(probability=True))])

train_x, train_y = load_training_set(feature_func=get_zernike_features)
pipe.fit(train_x, train_y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', SVC(probability=True))])

In [76]:
df = eval_results(image=im_cleanup, 
             boxes=detect_symbols(im_cleanup),
             feature_func=get_zernike_features,
             class_pipeline=pipe)

print(df.mean())
df.sort_values(by="recall", ascending=False)

clas         17.137931
precision     0.374466
recall        0.415580
dtype: float64


c:\Users\admpdi\Code\personal\CMP197\pid_detect\notebook\common.py:330: RuntimeWarning: invalid value encountered in long_scalars
  recall = np.sum(TP) / (np.sum(FN) + np.sum(TP))


,clas,precision,recall
6,9,1.000000,1.000000
8,11,1.000000,1.000000
12,15,0.750000,1.000000
7,10,1.000000,0.800000
16,19,0.800000,0.800000
2,4,0.750000,0.750000
4,6,1.000000,0.666667
15,18,1.000000,0.666667
3,5,0.333333,0.500000
5,7,1.000000,0.500000


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\admpdi\Code\su-visao\WPy64-3880\python-3.8.8.amd64\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\admpdi\Code\su-visao\WPy64-3880\python-3.8.8.amd64\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "c:\Users\admpdi\Code\personal\CMP197\pid_detect\.venv\lib\site-packages\matplotlib\backends\_backend_tk.py", line 489, in delayed_destroy
    self.window.destroy()
  File "C:\Users\admpdi\Code\su-visao\WPy64-3880\python-3.8.8.amd64\lib\tkinter\__init__.py", line 2312, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\admpdi\Code\su-visao\WPy64-3880\python-3.8.8.amd64\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\admpdi\Code\su-visao\WPy64

In [61]:
plt.tight_layout()

In [73]:
pipe = Pipeline([("classifier", sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, metric="hamming"),)])

ham_features = lambda x: get_square_thresh(x).ravel()
train_x, train_y = load_training_set(feature_func=ham_features)
pipe.fit(train_x, train_y)

Pipeline(steps=[('classifier', KNeighborsClassifier(metric='hamming'))])

In [74]:
eval_results(image=im_cleanup, 
             boxes=detect_symbols(im_cleanup),
             feature_func=ham_features,
             class_pipeline=pipe)

In [ ]:

fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

draw = im_cleanup.copy()

# Detect
# symbol_boxes = detect_symbols(im_cleanup)
symbol_boxes = np.stack(data["symbols"]["box"])
predictions = np.stack(data["symbols"]["class"]).astype(int)

# Draw results
draw_detections(draw, np.stack(symbol_boxes).reshape(-1,2,2), predictions)
axs[0].imshow(draw)
draw2 = draw_symbols(draw, thickness=8, color=(0,255,0))
axs[1].imshow(draw2)

In [ ]:
from mapcalc import calculate_map, calculate_map_range


gt_boxes = np.stack(data["symbols"]["box"])
gt_classes = np.stack(data["symbols"]["class"]).astype(int)
ground_truth_info={"boxes":gt_boxes,"labels":gt_classes}
pred_info={"boxes":symbol_boxes,"labels":predictions}#, "scores":confidences}

# calculate_map_range(ground_truth, predictions, 0.5,0.95,0.05)
calculate_map(ground_truth_info, pred_info, 0.5)